# Profile generator based on ARERA data

This notebook aims at generating time series scenarios based on ARERA data,
and the corresponding configuration file compatible with EnergyCommunity.jl

## Main inputs and imports

Inputs

In [ ]:
data_folder = "../data/ARERA"  # Folder containing the data
NAME_FILE_START = "dati prelievo orario"  # Initial file name to be used as filter
output_merged = "all_data.csv"  # Output file name

province = "Pisa"  # Province of analysis
weeks_per_month = 1  # Number of representative weeks per month
months_per_year = 12  # Number of months per year

add_randomness = true  # When true, a gaussian noise is added to the profiles in agreement to parameters in the config.
                        # Alternatively, no randomness is added
user_specific = false  # When true, each user of the EC will have a specific profile (including randomness if enabled)
                        # When false, all users of the same class will have the same profile

config_file = "../data/config_profile_generator.yaml"  # Name of the config file for the analysis

Imports

In [ ]:
using XLSX, YAML, CSV, DataFrames, Dates

## Read all input files

Read config file

In [ ]:
config = YAML.load_file(config_file)  # read config file

component_types = config["component_types"]
user_types = config["user_types"]
EC_scenarios = config["EC_scenarios"];

Read all data files

In [ ]:
fp = "C:/Users/Davide/Desktop/ECPreprocessing/data/ARERA/dati prelievo orario per provincia potenza6 anno 2022.xlsx"

"""
Reads the ARERA xlsx file and cleans it
"""
function read_arera_xlsx(fp)
    df_temp = DataFrame(XLSX.readtable(fp, 1))
    df_temp = filter(x->(x["Anno mese"] isa Date), df_temp)
    df_temp[!, :Mese] =  Dates.month.(df_temp[!, "Anno mese"])
    df_temp[!, :Orario] = parse.(Int, map(x->x[4:end], df_temp[!, :Orario]))
    rename!(df_temp, "Prelievo medio Orario Provinciale (kWh)"=>"Prelievo_kWh")
    return df_temp
end

In [ ]:
df_list_data = []

for (root, dirs, files) in walkdir(data_folder)
    for file in files
        if endswith(file, ".xlsx") && startswith(file, NAME_FILE_START)
            fp = joinpath(root, file)  # get the full path of the file
            println(fp)
            df = read_arera_xlsx(fp)  # read the data
            df_st = filter(x->x[:Provincia]==province, df)[:, Not(:Provincia)]
            push!(df_list_data, df_st)  # append the DataFrame to the list
        end
    end
end

# merge data
df_arera = vcat(df_list_data...)

first(df_arera, 5)

#### Create weekly time series for the relevant consumer types

In [ ]:
"""
    to_weekly_profile(df)

Given an ARERA dataframe with columns "Working Day" and "Orario",
it generates a weekly profile, considering the profiles for Saturday (SAB), Sunday (DOM) and weekdays (Giorno_feriale)
"""
function to_weekly_profile(df)
    df_m = copy(df)
    df_m[df_m[!, "Working Day"] .== "SAB", :Orario] .+= 5 * 24
    df_m[df_m[!, "Working Day"] .== "DOM", :Orario] .+= 6 * 24
    df_feriale = df_m[df_m[!, "Working Day"] .== "Giorno_feriale", Not("Working Day")]
    df_list = [df_m[!, Not("Working Day")]]
    for i = 1:4
        df_temp = copy(df_feriale)
        df_temp[!, :Orario] .+= i * 24
        push!(df_list, df_temp)
    end

    df_week = vcat(df_list...)
    sort!(df_week, :Orario)
    return df_week
end

#### Creates a yearly representative dataframe for each user type

In [ ]:
"""
    create_year_equivalent_profiles

This functions reads the dictionary of user profiles and returns a modified year-equivalent version.
"""
function create_year_equivalent_profiles(
    user_weekly_profiles, weeks_per_month, months_per_year,
)
    @assert (mod(12, months_per_year) == 0) "Parameter months_per_year must be a divisor of 12 months"
    @assert (1 <= weeks_per_month <= 4) "Parameter weeks_per_month must be within 1 and 4"

    eq_months = 1:months_per_year  # index of equivalent index month
    months_to_merge = Int(12 / months_per_year)  # number of months to merge per year

    custom_mean(x...) = .+(x...)/length(x)

    year_equivalent_profiles = Dict{String, Vector{Float64}}()
    for (u_name, u_value) in user_weekly_profiles
        # Aggregate months
        eq_month_profile = select(
            u_value,
            [
                Symbol.(["$(e+i-1)" for i in 1:months_to_merge]) => custom_mean => Symbol("$e")
                for e in eq_months
            ]...,
            copycols=false,
        )

        # if multiple weeks per month are selected, expand the profiles as appropriate
        if weeks_per_month > 1
            df_list = [eq_month_profile]
            for i = 2:weeks_per_month
                df_temp = copy(eq_month_profile)
                df_temp[!, :Orario] .+= 24*7
                push!(df_list, df_temp)
            end
            eq_month_profile = vcat(df_list...)
        end

        year_equivalent_profiles[u_name] = vcat(
            eachcol(eq_month_profile[!, ["$e" for e in eq_months]])...
        )
    end

    df_yearly = DataFrame(year_equivalent_profiles)
    df_yearly[!, :Orario] = 1:nrow(df_yearly)

    return df_yearly[!, ["Orario"; collect(keys(year_equivalent_profiles))]]
end

In [ ]:
user_weekly_profiles = Dict{String, DataFrame}()

FILTER_COLS = ["Mese", "Working Day", "Orario", "Prelievo_kWh"]

for (u_name, u_value) in user_types
    
    # filter the original df_arera dataframe to select only relevant info
    df_raw = filter(
        x->all(x[c]==v for (c,v) in u_value["filter_codes"]),
        df_arera,
    )[!, FILTER_COLS]

    gdf = groupby(df_raw, :Mese)  # groupby by month
    weekly_profiles_stacked = combine(gdf, to_weekly_profile)  # create weekly time series for each month
    weekly_profiles = unstack(weekly_profiles_stacked, :Mese, :Prelievo_kWh)  # create unstacked dataframe

    user_weekly_profiles[u_name] = weekly_profiles
end
user_weekly_profiles

Create DataFrame with yearly data by row and in columns the name of user types

In [ ]:
user_yearly_profiles = create_year_equivalent_profiles(user_weekly_profiles, weeks_per_month, months_per_year)
first(user_yearly_profiles, 5)

#### Generate profiles for the target ECs as specified in the config file

In [ ]:
for (s_name, s_value) in EC_scenarios
    
end